In [24]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D

In [25]:
def generate_cubes(x, y, z, size):
    cubes = []
    for xi in x:
        for yi in y:
            for zi in z:
                cube = [
                    [xi, yi, zi],
                    [xi + size, yi, zi],
                    [xi + size, yi + size, zi],
                    [xi, yi + size, zi],
                    [xi, yi, zi + size],
                    [xi + size, yi, zi + size],
                    [xi + size, yi + size, zi + size],
                    [xi, yi + size, zi + size]
                ]
                cubes.append(cube)
    return cubes



In [26]:
#create a tetrahedra grid
def generate_cube_tetrahedra(vertices):
    tetrahedra = []

    # Define the indices of the cube vertices
    cube_vertex_indices = list(range(8))

    # Define the indices of vertices for each tetrahedron
    tetrahedron_vertex_indices = [
            [0, 1, 2, 4],
            [1, 2, 4, 5],
            [2, 4, 5, 6],
            [0, 2, 3, 6],
            [0, 3, 6, 7],
            [0, 4, 6, 7]]


    # Create the tetrahedra
    for indices in tetrahedron_vertex_indices:
        tetrahedron = [vertices[i] for i in indices]
        tetrahedra.append(tetrahedron)

    return tetrahedra

In [27]:
def plot_tetrahedron(vertices):
    # Define the vertex positions of the tetrahedron
    x, y, z = zip(*vertices)

    # Create a mesh3d trace for the tetrahedron
    tetrahedron_trace = go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=[0, 1, 2, 0, 1, 3, 2, 3],
        j=[1, 2, 3, 3, 0, 2, 0, 1],
        k=[2, 3, 0, 1, 2, 3, 1, 0],
        color='lightblue',
        opacity=0.8
    )

    # Create a layout for the plot
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    # Create a figure and add the trace to it
    fig = go.Figure(data=[tetrahedron_trace], layout=layout)

    # Show the plot
    fig.show()



In [28]:
def plot_triangles(triangles):
    # Configuración de límites
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    # Crear trazas para los triángulos
    data = []
    for triangle_points in triangles:
        x = [point[0] for point in triangle_points]
        y = [point[1] for point in triangle_points]
        z = [point[2] for point in triangle_points]

        edges = go.Scatter3d(x=x + [x[0]], y=y + [y[0]], z=z + [z[0]], mode='lines', line=dict(width=2, color='black'))

        data.append(edges)

    fig = go.Figure(data=data, layout=layout)
    fig.show()

### Funciones a usar

In [29]:
def elipsoide(x, y, z, a = 3, b = 3, c = 3):
    # Function F that determines the value at each point
    # Modify this function according to your specific surface or volume
    return x**2/a + y**2/b + z**2/c - 1


In [30]:
def Sigmoidal(x, n = 0.5):
    return 2/(1 + math.exp(-n*x))-1

In [31]:
def cilindro(x,y,z, r = 1, S = Sigmoidal):
    return S(x**2 + y**2 - r)


In [32]:
def dona(x,y,z, C = cilindro, E = elipsoide, S  = Sigmoidal):
    return E(x,y,z)+C(x,y,z)- E(x,y,z)*C(x,y,z)

### Marching Squares


In [33]:
def marching_cases(tethaedra, function):

    vertices = []

    if len(tethaedra) != 4:
        return vertices

    values = []

    for point in tethaedra:
        x,y,z = point
        values.append(function(x,y,z)) # Evaluate the function at each point

    # Determine the configuration based on inside/outside status
    configuration = 0
    for i, value in enumerate(values):
        if value > 0:
            configuration |= 1 << i



    if configuration == 1:
        p1 = (tethaedra[0]+tethaedra[1])*0.5
        p2 = (tethaedra[0]+tethaedra[2])*0.5
        p3 = (tethaedra[0]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 2:
        p1 = (tethaedra[1]+tethaedra[0])*0.5
        p2 = (tethaedra[1]+tethaedra[2])*0.5
        p3 = (tethaedra[1]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 3:
        p1 = (tethaedra[0]+tethaedra[2])*0.5
        p2 = (tethaedra[1]+tethaedra[2])*0.5
        p3 = (tethaedra[0]+tethaedra[3])*0.5
        p4 = (tethaedra[1]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 4:
        p1 = (tethaedra[2]+tethaedra[0])*0.5
        p2 = (tethaedra[2]+tethaedra[1])*0.5
        p3 = (tethaedra[2]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 5:
        p1 = (tethaedra[0]+tethaedra[1])*0.5
        p2 = (tethaedra[2]+tethaedra[3])*0.5
        p3 = (tethaedra[0]+tethaedra[3])*0.5
        p4 = (tethaedra[2]+tethaedra[1])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 6:
        p1 = (tethaedra[1]+tethaedra[3])*0.5
        p2 = (tethaedra[1]+tethaedra[0])*0.5
        p3 = (tethaedra[2]+tethaedra[3])*0.5
        p4 = (tethaedra[2]+tethaedra[0])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 7:
        #print(tetrahedra[3]+tetrahedra[0])
        p1 = (tethaedra[3]+tethaedra[0])*0.5
        p2 = (tethaedra[3]+tethaedra[1])*0.5
        p3 = (tethaedra[3]+tethaedra[2])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 8:
        p1 = (tethaedra[3]+tethaedra[0])*0.5
        p2 = (tethaedra[3]+tethaedra[1])*0.5
        p3 = (tethaedra[3]+tethaedra[2])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 9:
        p1 = (tethaedra[0]+tethaedra[1])*0.5
        p2 = (tethaedra[0]+tethaedra[2])*0.5
        p3 = (tethaedra[3]+tethaedra[1])*0.5
        p4 = (tethaedra[3]+tethaedra[2])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 10:
        p1 = (tethaedra[1]+tethaedra[0])*0.5
        p2 = (tethaedra[1]+tethaedra[2])*0.5
        p3 = (tethaedra[3]+tethaedra[0])*0.5
        p4 = (tethaedra[3]+tethaedra[2])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 11:
        p1 = (tethaedra[2]+tethaedra[0])*0.5
        p2 = (tethaedra[2]+tethaedra[1])*0.5
        p3 = (tethaedra[2]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 12:
        p1 = (tethaedra[2]+tethaedra[1])*0.5
        p2 = (tethaedra[2]+tethaedra[0])*0.5
        p3 = (tethaedra[3]+tethaedra[1])*0.5
        p4 = (tethaedra[3]+tethaedra[0])*0.5
        vertices.append([p1, p2, p3])
        vertices.append([p1, p2, p4])
    elif configuration == 13:
        p1 = (tethaedra[1]+tethaedra[0])*0.5
        p2 = (tethaedra[1]+tethaedra[2])*0.5
        p3 = (tethaedra[1]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])
    elif configuration == 14:
        p1 = (tethaedra[0]+tethaedra[1])*0.5
        p2 = (tethaedra[0]+tethaedra[2])*0.5
        p3 = (tethaedra[0]+tethaedra[3])*0.5
        vertices.append([p1, p2, p3])

    return vertices


In [34]:
def marching_tetrahedra(vertex_positions, function):
    # Iterate through each tetrahedron
    vertex = []
    N = len(vertex_positions)
    for i in range(N):
        cube = vertex_positions[i]
        if len(cube) != 6:
            print("Error: cube is not 6 tetrahedra")
        for j in range(6):
            tetrahedra = cube[j]
            if len(tetrahedra) != 4:
                print("Error: tetrahedra is not 4 points")
        # Determine the configuration of the tetrahedron
            vertices = marching_cases(tetrahedra, function)
            if vertices != []:
                vertex.append(vertices)

    return vertex
    #plot_triangles(vertex)

### Ploteo

In [35]:
L = 4
steps = 30
step = 2 * L / steps

x = np.arange(-L, L, step)  # X-coordinates of the cubes
y = np.arange(-L, L, step)  # Y-coordinates of the cubes
z = np.arange(-L, L, step)  # Z-coordinates of the cubes

Cubos = generate_cubes(x, y, z, step)
tetraedros = []

for cube in Cubos:
    tetraedros.append(generate_cube_tetrahedra(cube))

tetraedros = np.array(tetraedros)

#### Veo si todo salio bien

In [36]:
N = len(tetraedros)

for i in range(N):
        cube = tetraedros[i]
        if len(cube) != 6:
            print("Error: cube is not 6 tetrahedra")
        for j in range(6):
            tetrahedra = cube[j]
            if len(tetrahedra) == 6:
                print("Error: tetrahedra is not 4 points")


In [37]:
V = marching_tetrahedra(tetraedros, cilindro)

In [38]:
vert = []
N = len(V)
for i in range(N):
    vert.append(V[i][0])

In [39]:
plot_triangles(vert)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed